# gu_toolkit: Efficient Guide, Compendium, and Applications

This notebook is an opinionated, practical guide for everyday toolkit use. It is organized for **fast onboarding** and **copy/paste utility**.

## How to use this notebook
1. Run the setup cell.
2. Jump to the section you need (quickstart, recipes, applications).
3. Copy a pattern and adapt it.

---

## Contents
- Quickstart
- Frequently used APIs (compendium)
- Parameter and numeric-expression patterns
- Introspection and debugging helpers
- Cool applications (interactive demos)


In [ ]:
# Find where the package root is
import sys
from pathlib import Path

try:
    _start = Path(__file__).resolve().parent
except NameError:
    _start = Path.cwd().resolve()

_pkg_root = _start
while _pkg_root != _pkg_root.parent and not (_pkg_root / "__init__.py").exists():
    _pkg_root = _pkg_root.parent
sys.path.insert(0, str(_pkg_root.parent))

In [ ]:
# Setup: Import toolkit
from gu_toolkit import *

# 1) Quickstart

## Minimal interactive figure

In [ ]:
fig = Figure(x_range=(-8, 8))
display(fig)

with fig:
    set_title("Quickstart: Interactive sine")
    plot(a*sin(b*x), x, id="wave", label="a*sin(bx)")

## Figure with some configuration

In [ ]:
fig = Figure(x_range=(-3*pi, 3* pi))
display(fig)

with fig:
    set_title("Quickstart: Interacting sine waves")
    plot(sin(b[1]*x), x, id="wave1", label="sin(b_1x)", color="blue", opacity=0.075, thickness=5)
    plot(sin(b[2]*x), x, id="wave2", label="sin(b_2x)", color="green", opacity=0.075, thickness=5)
    plot(sin(b[1]*x)*sin(b[2]*x), x, id="product_wave", label="sin(b_1x) sin(b_2x)", color="red", opacity=1)
    parameter(b[1], value=1, min=1, max=20, step=1)
    parameter(b[2], value=2, min=1, max=20, step=1)
   
#BUG: introduce alpha as a synonym for opacity, introduce width as a synonym for thickness

## Saving Figure code

Make sure to run the code to create the figure. Play around with it and decide on specific values of the parameters. Later `figure.to_code()` can be used to print out the code that will generate the same figure

In [ ]:
print(fig.to_code())
#BUG make fig.code a readonly property.
#BUG make fig.get_code(options) be the call that allows setting options
#BUG when generating code, call display(fig) instead of just fig. If using context code, use set_title inside the context. Also, display the figure immediately after defintion i.e. display(fig) follows fig=

## Plotting a purely numeric function or a python callable

The callable-first API supports plain Python functions as long as they can evaluate over NumPy arrays.


In [ ]:
import numpy as np

x = sp.Symbol("x")
fig_callable = Figure(x_range=(-6, 6), y_range=(-1.5, 1.5), sampling_points=500)
display(fig_callable)

with fig_callable:
    set_title("Numeric callable: damped oscillation")

    def damped_wave(x):
        return np.exp(-0.15 * x**2) * np.cos(3 * x)

    plot(damped_wave, x, id="damped", color="teal", label="exp(-0.15x^2) cos(3x)")


In [ ]:
x, A, k = sp.symbols("x A k")
fig_callable_params = Figure(x_range=(-8, 8), y_range=(-2.5, 2.5), sampling_points=600)
display(fig_callable_params)

with fig_callable_params:
    set_title("Callable with explicit vars + sliders")
    plot(lambda x, A, k: A * np.sin(k * x), x, vars=(x, A, k), id="param-wave", color="orange")
    parameter(A, value=1.2, min=0.0, max=2.5, step=0.1)
    parameter(k, value=1.0, min=0.5, max=4.0, step=0.1)


## Quickstart: adding non-trivial info cards

Info cards can mix static HTML with dynamic callback segments that re-evaluate on render and slider changes.
Use this pattern to show diagnostics, derived metrics, and runtime context next to your plot.


In [ ]:
x, a, b = sp.symbols("x a b")
fig_info = Figure(x_range=(-2*sp.pi, 2*sp.pi), y_range=(-3, 3), sampling_points=600)
display(fig_info)

with fig_info:
    set_title("Quickstart: plot + diagnostics info cards")
    plot(a*sp.sin(b*x), x, id="main-wave", label="a*sin(bx)", color="royalblue")
    parameter(a, value=1.0, min=0.2, max=2.0, step=0.1)
    parameter(b, value=1.0, min=0.5, max=4.0, step=0.1)

    info("""
    <b>Tips</b><br>
    • Move <code>a</code> to scale amplitude.<br>
    • Move <code>b</code> to change frequency.<br>
    """, id="usage-tips")

    def diagnostics_card(fig, ctx):
        p = fig.params.snapshot()
        amp = p["a"]
        freq = p["b"]
        period = (2*np.pi/freq) if freq else float("inf")
        est_energy = 0.5 * amp**2
        return (
            f"<b>Wave diagnostics</b><br>"
            f"reason: <code>{ctx.reason}</code><br>"
            f"a={amp:.2f}, b={freq:.2f}<br>"
            f"period≈{period:.3f}, ½a²≈{est_energy:.3f}"
        )

    info(["<hr>", diagnostics_card], id="wave-diagnostics")


## Using multiple tabs


In [ ]:
x, a, b = sp.symbols("x a b")
fig_tabs = Figure(x_range=(-6, 6), y_range=(-3, 3), sampling_points=400)
fig_tabs.add_view("phase", x_range=(-2*sp.pi, 2*sp.pi), y_range=(-3, 3), x_label="phase")

display(fig_tabs)

with fig_tabs:
    parameter(a, value=1.0, min=0.0, max=2.0, step=0.1)
    parameter(b, value=0.5, min=-2.0, max=2.0, step=0.1)

    plot(a*sp.sin(x) + b, x, id="shared-wave", view=("main", "phase"), color="crimson")
    with fig_tabs.view("phase"):
        plot(sp.cos(2*x), x, id="phase-only", color="purple")

    info("<b>Shared card:</b> visible on every view", id="shared-card")
    info("<b>Phase note:</b> scoped to the phase tab", id="phase-card", view="phase")


# 2) Compendium: Frequently used operations

BUG: Add significantly more documentation to these commands here
### Figure construction and ranges
- `Figure(...)`
- `get_x_range()`, `get_y_range()`
- `set_title(...)`

### Plot and parameter workflow
- `plot(...)`
- `parameter(symbol, ...)`
- `params[symbol]` / `params.snapshot()`
- `render()`

### Numeric expressions
- `numpify(expr, vars=...)`
- `DYNAMIC_PARAMETER` and `UNFREEZE`

### Symbolic parsing
- `parse_latex(latex_string)`


### B) Symbolic-to-numeric with dynamic parameters

Use this when you want to start from symbolic math and still keep interactivity.


In [ ]:
expr = parse_latex(r"A \sin(k x + \phi)")
expr.free_symbols

In [ ]:
f = numpify(expr, vars=['x', 'A', 'k', 'phi'])
f

## 4) Debugging and introspection helpers

These checks are useful when a notebook cell is not behaving as expected.


In [ ]:
with fig3:
    print("Current params:", params.snapshot())
print("Figure has", len(fig3.plots), "plot(s)")
print("Plot ids:", list(fig3.plots.keys()))

plot_obj = fig3.plots["symbolic-wave"]
print("Plot label:", plot_obj.label)
print("Has cached samples:", plot_obj.cached_samples is not None)

#BUG 
#---------------------------------------------------------------------------
#AttributeError                            Traceback (most recent call last)
#Cell In[21], line 8
#      6 plot_obj = fig3.plots["symbolic-wave"]
#      7 print("Plot label:", plot_obj.label)
#----> 8 print("Has cached samples:", plot_obj.cached_samples is not None)
#
#AttributeError: 'Plot' object has no attribute 'cached_samples'

## 6) Practical checklist

When prototyping in notebooks:
- Start with one plot and one parameter.
- Confirm `params.snapshot()` updates while sliders move.
- Increase `sampling_points` only when needed.
- Use clear `id=` values for plots you update repeatedly.

This notebook is intended as a durable daily-use reference.


## 7) Project-019: Tabbed multi-view workspace (phase 5/6 showcase)

This section demonstrates:
- creating multiple named views/tabs via `add_view(...)`,
- targeting plots and info cards to specific views,
- temporary view scoping with `with fig.view("...")`,
- snapshot/codegen support for multi-view state.


In [ ]:
x, a, b = sp.symbols("x a b")
fig_multi = Figure(x_range=(-6, 6), y_range=(-3, 3), sampling_points=400)
fig_multi.title = "Project-019 multi-view demo"

# Add an alternate view with independent default ranges/labels
fig_multi.add_view("frequency", x_range=(-12, 12), y_range=(-2, 2), x_label="ω", y_label="magnitude")

with fig_multi:
    # Shared parameters drive both views
    parameter(a, value=1.0, min=0.0, max=2.0, step=0.1)
    parameter(b, value=0.5, min=-2.0, max=2.0, step=0.1)

    # Plot in both views
    plot(a*sp.sin(x) + b, x, id="shared-wave", view=("main", "frequency"))

    # View-scoped plot via context manager
    with fig_multi.view("frequency"):
        plot(sp.cos(2*x), x, id="freq-only", color="purple")

    # Shared and view-scoped info cards
    info("<b>Shared card:</b> visible on every view", id="shared-card")
    info("<b>Frequency note:</b> only visible on 'frequency'", id="freq-card", view="frequency")

fig_multi
#BUG freq-only appears on both views
#BUG the visual of the tab layout is bad. There are tabs on top with some space below that contains empty space. Then below that there is a figure. Shouldn't the figure be IN the tab area?

In [ ]:
print("Active view:", fig_multi.active_view_id)
print("Views:", tuple(fig_multi.views.keys()))
print("shared-wave memberships:", fig_multi.plots["shared-wave"].views)

snap = fig_multi.snapshot()
print("snapshot.active_view_id:", snap.active_view_id)
print("snapshot view ids:", tuple(v.id for v in snap.views))
print("snapshot scoped cards:", [(c.id, c.view_id) for c in snap.info_cards])

code = fig_multi.to_code()
print("code contains add_view:", "fig.add_view('frequency'" in code)
print("code contains scoped info:", "view='frequency'" in code)